TEST 1: BASIC KERAS

This test tokenizes movie reviews from the keras IMDB set and tries to classify them as "negative" or "positive" reviews.

In [2]:
#This test provided by https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/ 
#From their paper http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.bib

#Importing the imdb dataset and essential libraries
from keras.datasets import imdb
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, RNN
from keras import optimizers
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
import numpy as np
import os, sys, glob
#Fix a random seed
np.random.seed(13)

##Preprocessing
#load text 
filename = "../resources/switchboard_scrubbed3/dataset_small_test_file.txt"
raw_text = open(filename).read()

split_words = text_to_word_sequence(raw_text, lower=False)

# estimate vocab size
vocab = set(split_words)
vocab_size = len(vocab)

tokenizer = Tokenizer(num_words=vocab_size)

#fit the tokenizer on the text 
tokenizer.fit_on_texts(raw_text)

# integer encode text
encoded_docs = tokenizer.texts_to_matrix(raw_text, mode='count')

#Load only the top 20 words. 10 for testing, 10 for training.
word_pool_size = 20
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=word_pool_size)
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

#Creating the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(word_pool_size, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
#Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

TEST 2: GENERATING TEXT

This test loads the full text of Alice in Wonderland, tokenizes the text, and outputs a generated paragraph.

In [1]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=1, batch_size=128, callbacks=callbacks_list) # SET THIS BACK TO 20 EPOCHS FOR MORE RIGOROUS TESTING

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Using TensorFlow backend.


AttributeError: module 'tensorflow.python.training.checkpointable' has no attribute 'CheckpointableBase'

This test provided by https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
From their paper http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.bib